<h1> Processing of the yeast data <span class="tocSkip"></span></h1>

# Introduction

The data needs to be processed to reduce the memory required for training, particularly in cases such as statistical or tree-based methods that need to reduce dimensionality to achieve an efficient balance between time, memory, and performance during training. Additionally, various datasets will be generated with different percentages of missing data to evaluate imputation performance across these different cases.

In [3]:
import numpy as np
import pandas as pd  
from matplotlib import pyplot as plt


# Dataset

## Loading data

In [4]:
input_name = '../data/yeast_genotype_train.txt'
df_train = pd.read_csv('../data/raw/yeast_genotype_train.txt', sep='\t', index_col=0)
df_test = pd.read_csv('../data/raw/yeast_genotype_test.txt', sep='\t', index_col=0)


In [5]:
#dimensionality in originas sets
df_train.shape, df_test.shape

((3513, 28220), (877, 28220))

In [6]:
#examples of data
df_train.head(10)

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_02,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_03,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
01_07,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_08,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_09,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_10,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2


## Reducing dimensionality with MCA
Análisis de Correspondencia Múltiple (MCA) es una técnica que te permitirá reducir la dimensionalidad de tus datos categórico

In [15]:
%pip install prince
import pandas as pd
import prince

# Supongamos que tu df_train es un DataFrame de pandas con las variantes genéticas
# Si no tienes los datos categóricos en tipo `category`, conviértelos
df_train = df_train.astype('category')

# Aplicar Análisis de Correspondencia Múltiple (MCA)
mca = prince.MCA(
    n_components=1000,  # Número de componentes principales
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)


Note: you may need to restart the kernel to use updated packages.


In [16]:
# Ajustar el MCA a los datos de entrenamiento
mca = mca.fit(df_train)

# Transformar los datos originales en el espacio de baja dimensionalidad
df_train_mca = mca.transform(df_train)

# Ver las nuevas dimensiones
print(df_train_mca.head())

            0         1         2         3         4         5         6    \
SAMID                                                                         
01_01 -0.080395 -0.096950  0.006693  0.100443  0.223722 -0.057233 -0.085609   
01_02 -0.149023  0.148298  0.006746 -0.168970  0.017002  0.170052  0.104529   
01_03  0.296869 -0.035685  0.142189 -0.131911  0.090818 -0.082309  0.071686   
01_04 -0.070041 -0.013868 -0.124240  0.202737 -0.314865 -0.037786 -0.077825   
01_06  0.030780 -0.293393  0.061259  0.128743  0.151630 -0.002875  0.147508   

            7         8         9    ...       990       991       992  \
SAMID                                ...                                 
01_01  0.133797  0.004906  0.126534  ...  0.005638  0.002499  0.001552   
01_02 -0.026113  0.350336 -0.079120  ...  0.002494  0.002739 -0.008175   
01_03 -0.003067 -0.280643 -0.113360  ... -0.000609 -0.002613  0.000978   
01_04 -0.104239 -0.064355  0.070407  ...  0.000171 -0.004312 -0.001367   
01

In [ ]:
from sklearn.model_selection import train_test_split
df_train_split, df_val_split = train_test_split(df_train, test_size=0.2, random_state=42)

# Method

In [20]:
# Convertir df_test a categoría si es necesario
df_test = df_test.astype('category')

# Aplicar la misma transformación MCA al conjunto de test
df_test_mca = mca.transform(df_test)

# Introducir valores 0 sintéticos de manera aleatoria en el conjunto de test
import numpy as np

def introduce_missingness(data, missing_perc=0.1):
    """
    Introduce valores faltantes (0) en un porcentaje determinado de celdas en el DataFrame.
    """
    data_missing = data.copy()

    # Asegurarse de que el valor '0' esté en las categorías de todas las columnas categóricas
    for col in data_missing.select_dtypes(['category']).columns:
        data_missing[col] = data_missing[col].cat.add_categories([0])

    # Calcular el número de valores a modificar
    n_samples, n_features = data_missing.shape
    n_missing = int(np.floor(n_samples * n_features * missing_perc))

    # Seleccionar aleatoriamente las posiciones donde se introducirán los valores 0
    missing_indices = np.random.choice(n_samples * n_features, n_missing, replace=False)

    # Introducir los valores faltantes (0) en las posiciones seleccionadas
    for index in missing_indices:
        i = index // n_features
        j = index % n_features
        data_missing.iloc[i, j] = 0  # Introducir 0s como valores faltantes

    return data_missing


df_test_mca_missing= introduce_missingness(df_test, missing_perc=0.1)

KeyboardInterrupt: 